# Exercise 7 Census Reporter 
#Annie Ludlow 

In [1]:
import requests
import csv
from time import sleep
import pandas as pd
import numpy as np
import warnings

#prevent error of api 
from tenacity import retry

#import csv file 
inputfile = open('msas.csv', 'r')
msas = csv.reader(inputfile)

#creating lists to store values from censusreporter
allcbsas = []
allnames = []
incomedemocount = []
hispanicdemocount = []
agedemocount = []

#tables 
tableids = ['B19001', 'B01001', 'B03002']
tableidstring = ','.join(tableids)
tableids

#parameter 1 HHincome 50k-74.9k
#parameter 2 18-34
#parameter 3 large hispanic populations 

#force retry 
@retry
def getdata(url):
    data = requests.get(url).json()
    return(data)
    
requesturl = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03002&geo_ids=310|01000US' 

loadedjson = requests.get(requesturl)


#for loop
count = 0
for msa in msas:
    count += 1
    #skipping header row of csv file
    if count != 1:

        #pull out cbsa, append to list
        cbsa = msa[0]
        
        allcbsas.append(cbsa)
        #pull out name of cbsa, append to list
        name = msa[1]
        allnames.append(name)
        
        #create url for a specific msa
        url = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US%s' % (cbsa)
        print(url)
        #get data for cbsa
        data = getdata(url)


        # income (#1)
        # parse through the dictionary returned and get just the value we need
        inc = data['data']['31000US%s' % (cbsa)]['B19001']['estimate']
        inc50 = inc['B19001011']
        inc60 = inc['B19001012']
        totalincomecount = inc50 + inc60
        #write that value to the corresponding list of values
        incomedemocount.append(totalincomecount)
        
        # age (#2)
        # parse through the dictionary returned and get just the value we need
        age = data['data']['31000US%s' % (cbsa)]['B01001']['estimate']
        ageM18_19 = age['B01001007']
        ageM20 = age['B01001008']
        ageM21 = age['B01001009']
        ageM22_24 = age['B01001010']
        ageM25_29 = age['B01001011']
        ageM30_34 = age['B01001012']
        ageF18_19 = age['B01001031']
        ageF20 = age['B01001032']
        ageF21 = age['B01001033']
        ageF22_24 = age['B01001034']
        ageF25_29 = age['B01001035']
        ageF30_34 = age['B01001036']
        ageT = ageM18_19 + ageM20 + ageM21 + ageM22_24 + ageM25_29 + ageM30_34 \
            + ageF18_19 + ageF20 + ageF21 + ageF22_24 + ageF25_29 + ageF30_34
        #write that value to the corresponding list of values
        agedemocount.append(ageT)

        # hispanic (#3)
        # parse through the dictionary returned and get just the value we need
        pop_hisp = data['data']['31000US%s' % (cbsa)]['B03003']['estimate']['B03003003']
        #write that value to the corresponding list of values
        hispanicdemocount.append(pop_hisp)

#create a new dataframe
df = pd.DataFrame()

#write the lists to columns, one by one
df['cbsa'] = allcbsas
df['name'] = allnames
df['income'] = incomedemocount
df['hispanic'] = hispanicdemocount
df['age'] = agedemocount



#find out the max values
#max income
print('Q1: max income')
print('-------------------------')
df.iloc[df.income.argmax()]

#max age
print('Q2: max age')
print('-------------------------')
df.iloc[df.age.argmax()]

#max hispanic
print('Q3: max hispanic')
print('-------------------------')
df.iloc[df.hispanic.argmax()]


df['meanincome'] = df['income']/df['income'].max()
df['meanhispanic'] = df['hispanic']/df['hispanic'].max()                
df['meanage'] = df['age']/df['age'].max()  

#mean of the 3 means 
df['totalmean'] = (df['meanincome'] + df['meanhispanic'] + df['meanage']) / 3

#highest mean wins
print('Q4: highest cumulative mean')
print('-------------------------')
df.iloc[df.totalmean.argmax()]


https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US10180
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US10420
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US10500
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US10540
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US10580
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US10740
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US10780
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US10900
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US11020
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B1

https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US17980
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US18020
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US18140
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US18580
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US18700
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US18880
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US19060
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US19100
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US19140
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B1

https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US26380
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US26420
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US26580
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US26620
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US26820
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US26900
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US26980
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US27060
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US27100
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B1

https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34060
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34100
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34580
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34620
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34740
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34820
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34900
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34940
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US34980
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B1

https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US42220
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US42340
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US42540
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US42660
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US42680
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US42700
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US43100
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US43300
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B19001,B01001,B03003&geo_ids=31000US43340
https://api.censusreporter.org/1.0/data/show/latest?table_ids=B1

/Users/annieludlow/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:121: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
/Users/annieludlow/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:126: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
/Users/annieludlow/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:131: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Q4: highest cumulative mean
-------------------------


/Users/annieludlow/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:147: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


cbsa                                            35620
name            New York-Newark-Jersey City, NY-NJ-PA
income                                    1.03819e+06
hispanic                                   4.9944e+06
age                                       4.80694e+06
meanincome                                          1
meanhispanic                                 0.828053
meanage                                             1
totalmean                                    0.942684
Name: 249, dtype: object